<a href="https://colab.research.google.com/github/karthikm-0/csc2515-project-stackoverflow/blob/main/BERT_QA_Pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.6MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
     |████████████████████████████████| 2.9MB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9740bd3af806637f355be3ab8d6cb3e009b957c9e691ab1bf8ec4ee704b59f76
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive
drive.mount('/content/drive')
project_folder = "./drive/My Drive/csc2515-project/"

Mounted at /content/drive


In [8]:
import pandas as pd
import os
import numpy as np

# Load dataset
questions_answers = pd.read_csv(os.path.join(project_folder, "questions_answers.csv"))

#questions_answers_test = questions_answers.sample(frac=0.2)

print(questions_answers.head(5).to_string())

   Unnamed: 0  QId   QAskerId         QCreationDate QClosedDate  QScore                              QTitle                                                                                                                                                                                                                                                                  QBody     Id  OwnerUserId          CreationDate  ParentId  Score                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [9]:
# Perform tokenization as BERT needs it in a specific format
def tokenize(data, type, seq_length=128):
  from transformers import BertTokenizer
  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

  questions_input_ids_list = []
  questions_attention_masks_list = []
  answers_input_ids_list = []
  answers_attention_masks_list = []
  scores_list = []

  for (idx, row) in data.iterrows():
      tokenize = tokenizer([row.QBody, row.Body], max_length=seq_length, padding='max_length', truncation=True) )
      tokenize_q = tokenizer(row.QBody, max_length=seq_length, padding='max_length', truncation=True)
      tokenize_a = tokenize_q = tokenizer(row.Body, max_length=seq_length, padding='max_length', truncation=True)

      questions_input_ids_list.append(tokenize_q['input_ids'])
      questions_attention_masks_list.append(tokenize_q['attention_mask'])

      answers_input_ids_list.append(tokenize_a['input_ids'])
      answers_attention_masks_list.append(tokenize_a['attention_mask'])

      if type == 'train':
        scores_list.append(row.Score)

  # TFBertForSequenceClassification requires Numpy arrays
  questions_input_ids_list = np.asarray(questions_input_ids_list)
  questions_attention_masks_list = np.array(questions_attention_masks_list)

  answers_input_ids_list = np.asarray(answers_input_ids_list)
  answers_attention_masks_list = np.array(answers_attention_masks_list)

  scores_list = np.array(scores_list)

  return questions_input_ids_list, questions_attention_masks_list, answers_input_ids_list, answers_attention_masks_list, scores_list

In [7]:
from sklearn.model_selection import train_test_split

# Keep test for after we train the model and use training data only for train and val
questions_answers_train, questions_answers_test, = train_test_split(questions_answers, test_size=0.2)
print(questions_answers_train.shape)

q_input_ids, q_attention_masks, a_input_ids, a_attention_masks, scores = tokenize(questions_answers_train, type='train')

(96348, 14)


In [18]:
# Save the tokens for future use
import pickle

pickle_q_inp_path = os.path.join(project_folder, "BERT/tokenized/bert_inp_q.pkl")
pickle_q_mask_path = os.path.join(project_folder, "BERT/tokenized/bert_mask_q.pkl")

pickle_a_inp_path = os.path.join(project_folder, "BERT/tokenized/bert_inp_a.pkl")
pickle_a_mask_path = os.path.join(project_folder, "BERT/tokenized/bert_mask_a.pkl")

pickle_score_path = os.path.join(project_folder, "BERT/tokenized/bert_scores.pkl")

pickle.dump((q_input_ids),open(pickle_q_inp_path,'wb'))
pickle.dump((q_attention_masks),open(pickle_q_mask_path,'wb'))

pickle.dump((a_input_ids),open(pickle_a_inp_path,'wb'))
pickle.dump((a_attention_masks),open(pickle_a_mask_path,'wb'))

pickle.dump((scores),open(pickle_score_path,'wb'))

In [31]:
from sklearn.model_selection import train_test_split

train_q_input_ids, val_q_input_ids, train_q_masks, val_q_masks, train_a_input_ids, val_a_input_ids, train_a_masks, val_a_masks, train_scores, val_scores = train_test_split(q_input_ids, q_attention_masks, a_input_ids, a_attention_masks, scores, test_size=0.2)


print(train_q_input_ids.shape, val_q_input_ids.shape, train_q_masks.shape, val_q_masks.shape, 
      train_a_input_ids.shape, val_a_input_ids.shape, train_a_masks.shape, val_a_masks.shape,
      train_scores.shape, val_scores.shape)

(77078, 128) (19270, 128) (77078, 128) (19270, 128) (77078, 128) (19270, 128) (77078, 128) (19270, 128) (77078,) (19270,)


In [33]:
# Train model
from transformers import BertConfig, TFBertForSequenceClassification

# Load Model
import tensorflow as tf
import keras


#print(tensorflow.__version__)
from transformers import BertConfig
model_config = BertConfig.from_pretrained('bert-base-uncased')
model_config.num_labels = 1

from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=model_config)

loss = tf.keras.losses.MeanSquaredError
metric = tf.keras.metrics.MeanSquaredError('mse')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
print(model.summary())

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_75']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  769       
Total params: 109,483,009
Trainable params: 109,483,009
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#model.fit([train_x, train_mask], train_y, batch_size=32, epochs=2, validation_data=([test_x, test_mask], test_y), callbacks=checkpoint)
model.fit()